## 1: load water levels and coords

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sys
sys.path.append("..")

In [3]:
import pandas as pd
# import geopandas as gpd
import numpy as np
import os

import utils
from features.meteo import DirMeteoLoader, DirMeteoPreprocessor, DirMeteoDropper, DtBuilder

from importlib import reload

In [4]:
root_dir = "../../"
data_dir = os.path.join(root_dir, "datasets/")
processed_data_dir = os.path.join(data_dir, "processed_data/")

water_levels_path = os.path.join(data_dir, "hydro_2018-2020/new_data_all.csv")
posts_path = os.path.join(processed_data_dir, "asunp.pkl")

meteo_dir = os.path.join(data_dir, "meteo_new/")

In [5]:
meteo_loader = DirMeteoLoader(meteo_dir)
meteo = meteo_loader.load()
meteo.head(3)

,stationNumber,year,month,day,time,localYear,localMonth,localDay,localTimePeriod,timePeriodNum,...,dewpointTemperatureQuality,pressure,pressureQuality,pressureReducedToMeanSeaLevel,pressureReducedToMeanSeaLevelQuality,characteristicOfPressureTendency,characteristicOfPressureTendencyQuality,HourPressureChange3,HourPressureChange3Quality,stationId
52595,30879,1984,1,1,0,1984,1,1,3,4,...,0,951.400024,0,1038.599976,0,2.0,0,0.4,0,5131961
52596,30879,1984,1,1,3,1984,1,1,6,5,...,0,951.700012,0,1037.500000,0,2.0,0,0.3,0,5131961
52597,30879,1984,1,1,6,1984,1,1,9,6,...,0,951.200012,0,1035.500000,0,8.0,0,0.5,0,5131961


In [5]:
water_levels = pd.read_csv(water_levels_path, sep=";")
posts = utils.load_pickle(posts_path)

water_levels = utils.reduce_memory_usage(water_levels)
posts = utils.reduce_memory_usage(posts)

water_levels.head(3)
posts.head(3)

,time,max_level,identifier
0,2020-10-01 00:00:00,232.0,5116
1,1986-06-20 00:00:00,278.0,5292
2,1986-06-22 00:00:00,67.0,6022


,foId,pgid,pagr,paer,codBasin,station_id,kod1,kod2,kod3,agro,...,cgms,pn,stStatus,rv,pmet,codeSubject,ter,parentNpSn,kto,geometry
0,8.0,2.0,0.0,0.0,177,<NA>,639,445,000,NaN,...,999,100.0,open,0,0.0,14,98,5043.0,53,POINT (141.36667 63.33333)
1,8.0,4.0,0.0,0.0,177,<NA>,639,151,000,NaN,...,999,20.0,open,0,0.0,14,98,5049.0,53,POINT (144.30000 64.44667)
2,8.0,2.0,0.0,0.0,200,2224,619,000,000,NaN,...,999,28.0,open,0,0.0,41,30,5333.0,53,POINT (160.83000 56.81000)


## 2: form water_levels, manipulating with indexes 

In [12]:
pd.to_datetime(water_levels["time"], yearfirst=True)

0           1
1          20
2          22
3          23
4          22
           ..
2231614    15
2231615    17
2231616    17
2231617    14
2231618    15
Name: time, Length: 2231619, dtype: int64

In [13]:
water_datetime_date = pd.to_datetime(water_levels["time"], yearfirst=True).dt.date
water_levels["time"] = water_datetime_date

water_levels.rename(columns={"time": "date", "identifier": "id"}, inplace=True)

water_levels.sort_values(by=["id", "date"], inplace=True)
water_levels.set_index(["id", "date"], inplace=True)

water_levels.head()

max_level
id   date                 
5001 1984-01-01      258.0
     1984-01-02      255.0
     1984-01-03      252.0
     1984-01-04      248.0
     1984-01-05      244.0

## 3: get coords of hydrostations

In [62]:
def object_cols2num(df):
    """Tries to convert each objct col to num dtype, if error occures for
    some col, it's returned not modified"""
    object_cols_mask = (df.dtypes == "O").values
    object_cols = df.columns[object_cols_mask]
    
    for col in object_cols:
        df[col] = df[col].astype(np.float32, errors="ignore")
    return df

posts = object_cols2num(posts)

In [63]:
def coords_by_col_val(post_df, id_colname, needed_values):
    id_col_vals = post_df[id_colname]
    needed_rows_mask = id_col_vals.isin(needed_values)
    needed_rows = post_df[needed_rows_mask.values].reset_index(drop=True)
    
    coords_with_id_col = needed_rows[[id_colname, "lon", "lat"]]
    return coords_with_id_col

In [66]:
needed_hydro_ids = water_levels.reset_index()["id"].unique()
hydro_coords = coords_by_col_val(posts, "gidro", needed_hydro_ids)
hydro_coords.head(2)

,gidro,lon,lat
0,5674.0,132.80,48.72
1,5216.0,132.01,44.06


In [67]:
hydro_coords.drop_duplicates(subset="gidro", inplace=True)
hydro_coords.rename(columns={"gidro": "id"}, inplace=True)
hydro_coords.set_index("id", inplace=True)

## 4: meteo processing

In [6]:
meteo.head(2)

,stationNumber,year,month,day,time,localYear,localMonth,localDay,localTimePeriod,timePeriodNum,...,dewpointTemperatureQuality,pressure,pressureQuality,pressureReducedToMeanSeaLevel,pressureReducedToMeanSeaLevelQuality,characteristicOfPressureTendency,characteristicOfPressureTendencyQuality,HourPressureChange3,HourPressureChange3Quality,stationId
52595,30879,1984,1,1,0,1984,1,1,3,4,...,0,951.400024,0,1038.599976,0,2.0,0,0.4,0,5131961
52596,30879,1984,1,1,3,1984,1,1,6,5,...,0,951.700012,0,1037.500000,0,2.0,0,0.3,0,5131961


In [7]:
dt_builder = DtBuilder()
meteo_dropper = DirMeteoDropper()

In [9]:
preprocessor = DirMeteoPreprocessor(meteo, meteo_dropper, dt_builder)
meteo_processed = preprocessor.preprocess()
meteo_processed.head()

,stationNumber,cloudCoverTotal,windSpeed,totalAccumulatedPrecipitation,soilTemperature,airTemperature,minimumTemperatureAtHeightAndOverPeriodSpecified,maximumTemperatureOverPeriodSpecified,relativeHumidity,dewpointTemperature,pressure,pressureReducedToMeanSeaLevel,stationId,datetime,windAngleX,windAngleY
52595,30879,0.0,0.0,0.0,NaN,-33.700001,-33.700001,-32.099998,85.0,-35.299999,951.400024,1038.599976,5131961,1984-01-01 09:00:00,0.0,0.0
52596,30879,0.0,0.0,0.0,NaN,-29.700001,-33.700001,-29.700001,85.0,-31.500000,951.700012,1037.500000,5131961,1984-01-01 12:00:00,0.0,0.0
52597,30879,0.0,0.0,0.0,NaN,-26.000000,-29.700001,-25.799999,83.0,-27.700001,951.200012,1035.500000,5131961,1984-01-01 15:00:00,0.0,0.0
52598,30879,0.0,0.0,0.0,NaN,-28.100000,-28.100000,-25.900000,85.0,-29.799999,951.099976,1036.099976,5131961,1984-01-01 18:00:00,0.0,0.0
52599,30879,0.0,0.0,0.0,NaN,-31.000000,-31.000000,-28.100000,86.0,-32.799999,951.400024,1037.500000,5131961,1984-01-01 21:00:00,0.0,0.0


In [11]:
meteo_processed.set_index(["stationNumber", "datetime"], inplace=True)

In [15]:
date = meteo_processed.reset_index()["datetime"].dt.date
meteo_processed.groupby(["stationNumber", date.values]).agg(np.nanmean)

cloudCoverTotal  windSpeed  \
stationNumber                                          
30673         1984-01-01            6.800      0.200   
              1984-01-02            1.125      0.500   
              1984-01-03            0.750      0.375   
              1984-01-04            0.000      0.250   
              1984-01-05            2.000      0.125   
...                                   ...        ...   
31931         2020-03-27            7.000      1.875   
              2020-03-28            2.375      3.000   
              2020-03-29            1.000      3.875   
              2020-03-30            1.000      4.250   
              2020-03-31            4.750      4.250   

                          totalAccumulatedPrecipitation  soilTemperature  \
stationNumber                                                              
30673         1984-01-01                            0.0       -30.799999   
              1984-01-02                            0.0       -26.750000   
              1984-01-03                            0.0       -32.400002   
              1984-01-04                            0.0       -33.000000   
              1984-01-05                            0.0       -31.000000   
...                                                 ...              ...   
31931         2020-03-27                            0.8         0.550000   
              2020-03-28                            0.0         0.375000   
              2020-03-29                            0.0         3.875000   
              2020-03-30                            0.0         6.425000   
              2020-03-31                            0.0         5.900000   

                          airTemperature  \
stationNumber                              
30673         1984-01-01      -27.380001   
              1984-01-02      -27.725000   
              1984-01-03      -30.200001   
              1984-01-04      -30.087500   
              1984-01-05      -31.512501   
...                                  ...   
31931         2020-03-27        1.450000   
              2020-03-28       -1.025000   
              2020-03-29        3.250000   
              2020-03-30        7.487500   
              2020-03-31        8.825000   

                          minimumTemperatureAtHeightAndOverPeriodSpecified  \
stationNumber                                                                
30673         1984-01-01                                        -30.120001   
              1984-01-02                                        -30.900000   
              1984-01-03                                        -32.125000   
              1984-01-04                                        -31.862499   
              1984-01-05                                        -33.087502   
...                                                                    ...   
31931         2020-03-27                                          0.900000   
              2020-03-28                                         -1.912500   
              2020-03-29                                          1.312500   
              2020-03-30                                          5.687500   
              2020-03-31                                          6.637500   

                          maximumTemperatureOverPeriodSpecified  \
stationNumber                                                     
30673         1984-01-01                             -24.639999   
              1984-01-02                             -24.675001   
              1984-01-03                             -27.662500   
              1984-01-04                             -27.637499   
              1984-01-05                             -29.375000   
...                                                         ...   
31931         2020-03-27                               3.387500   
              2020-03-28                               0.125000   
              2020-03-29                          